# Producer en Python

Ce script utilise uniquement la bibliothèque kafka-python, sans passer par Spark.
Il sert à vérifier la connectivité avec le broker Kafka et à publier un message sur le topic test.

**Vous pouvez retrouver vos messages :**
- [Control Center](http://localhost:9021/clusters/)
- Section Topic -> candidate-signup



## Installation
kafka-python : bibliothèque en Python pour se connecter à Kafka

In [1]:
pip install kafka-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kafka import KafkaProducer
import json
import uuid
import random
from datetime import datetime, timezone
import time

In [3]:
# --------- Constantes -----------------
BROKER = "broker:29092"
TOPIC = "candidate-signup"

In [4]:
# --------- Producteur Kafka -----------
producer = KafkaProducer(
    bootstrap_servers=[BROKER],
    value_serializer=lambda v: json.dumps(v).encode("utf-8")
)

In [5]:
# --------- Données fixes --------------
USERS = [
    ("Romain", "Vasseur"),
    ("Adrien", "Vossough"),
    ("Emilie", "Bardin"),
    ("Luigi", "Menez"),
    ("Francois", "Salmon"),
    ("Alice", "Martin"),
    ("Claire", "Dubois"),
    ("David", "Petit")
]

CITIES = ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux", "Nantes", "Lille", "Nice"]
COUNTRIES = ["France", "Belgique", "Suisse"]

In [ ]:
# --------- Génération et envoi --------
def generate_random_user():
    first_name, last_name = random.choice(USERS)
    user_id = str(uuid.uuid4())
    email = f"{first_name.lower()}.{last_name.lower()}{random.randint(1,9999)}@example.com"
    city = random.choice(CITIES)
    country = random.choice(COUNTRIES)
    timestamp = datetime.now(timezone.utc).isoformat()
    return {
        "event_type": "user_signup",
        "user_id": user_id,
        "first_name": first_name,
        "last_name": last_name,
        "email": email,
        "city": city,
        "country": country,
        "registered_at": timestamp
    }

print(f"Envoi continu d’utilisateurs aléatoires sur le topic '{TOPIC}'… (Ctrl+C pour arrêter)")

try:
    while True:
        event = generate_random_user()
        producer.send(TOPIC, value=event)
        # décommenter et lancer pour afficher les logs
        # print(f"> envoyé : {event['first_name']} {event['last_name']} ({event['city']}, {event['country']})")
        
        time.sleep(1)
except KeyboardInterrupt:
    print("\nArrêt manuel demandé.")
finally:
    producer.flush()
    producer.close()

Envoi continu d’utilisateurs aléatoires sur le topic 'candidate-signup'… (Ctrl+C pour arrêter)
